In [1]:
import ROOT
import os,json
import ATLAS_table as ATLAS

Welcome to JupyROOT 6.28/04


In [2]:

ATLAS.SignalRegion


[('SR1', 'HTlep>200', 200, 2000, 'HTlep'),
 ('SR2', 'HTlep>500', 500, 2000, 'HTlep'),
 ('SR3', 'HTlep>800', 800, 2000, 'HTlep'),
 ('SR4', 'PTmin>50', 50, 2000, 'pTmin'),
 ('SR5', 'PTmin>100', 100, 2000, 'pTmin'),
 ('SR6', 'PTmin>150', 150, 2000, 'pTmin'),
 ('SR7', 'MET>0 & HTjets<150', 0, 2000, 'MET_HTb150'),
 ('SR8', 'MET>100 & HTjets<150', 100, 2000, 'MET_HTb150'),
 ('SR9', 'MET>200 & HTjets<150', 200, 2000, 'MET_HTb150'),
 ('SR10', 'MET>300 & HTjets<150', 300, 2000, 'MET_HTb150'),
 ('SR11', 'MET>0 & HTjets>150', 0, 2000, 'MET_HTa150'),
 ('SR12', 'MET>100 & HTjets>150', 100, 2000, 'MET_HTa150'),
 ('SR13', 'MET>200 & HTjets>150', 200, 2000, 'MET_HTa150'),
 ('SR14', 'MET>300 & HTjets>150', 300, 2000, 'MET_HTa150'),
 ('SR15', 'Meff>600', 600, 2000, 'Meff'),
 ('SR16', 'Meff>1000', 1000, 2000, 'Meff'),
 ('SR17', 'Meff>1500', 1500, 2000, 'Meff'),
 ('SR18', 'Meff>0 & MET>100', 0, 2000, 'Meff_METa100'),
 ('SR19', 'Meff>600 & MET>100', 600, 2000, 'Meff_METa100'),
 ('SR20', 'Meff>1200 & MET>10

In [3]:
ATLAS_SR={}
for item in ATLAS.SignalRegion:    
    ATLAS_SR[item[0]]=item[1]

print(ATLAS_SR)    

{'SR1': 'HTlep>200', 'SR2': 'HTlep>500', 'SR3': 'HTlep>800', 'SR4': 'PTmin>50', 'SR5': 'PTmin>100', 'SR6': 'PTmin>150', 'SR7': 'MET>0 & HTjets<150', 'SR8': 'MET>100 & HTjets<150', 'SR9': 'MET>200 & HTjets<150', 'SR10': 'MET>300 & HTjets<150', 'SR11': 'MET>0 & HTjets>150', 'SR12': 'MET>100 & HTjets>150', 'SR13': 'MET>200 & HTjets>150', 'SR14': 'MET>300 & HTjets>150', 'SR15': 'Meff>600', 'SR16': 'Meff>1000', 'SR17': 'Meff>1500', 'SR18': 'Meff>0 & MET>100', 'SR19': 'Meff>600 & MET>100', 'SR20': 'Meff>1200 & MET>100'}


In [4]:
def get_limits(filename,mass,SR):
    with open(filename,'r') as infile:
        limit_results = json.load(infile)
            
        limits=[]
        median   = limit_results[f'{SR}_nom__Mass{mass}'][f'{mass}'][1]
        theory   = limit_results[f'{SR}_nom__Mass{mass}'][f'{mass}'][2]
        uncUp    = limit_results[f'{SR}_up__Mass{mass}'][f'{mass}'][1]
        uncDown  = limit_results[f'{SR}_down__Mass{mass}'][f'{mass}'][1]        

        limits.extend([median,uncUp,uncDown,theory])
        
        return limits
        
        
def get_UpperLimit(values,SR):
    
    labels=[f'M{i}' for i in values]
    
    
    N = len(labels)
    unc     = ROOT.TGraph(2*N)     # unc
    median  = ROOT.TGraph(N)       # median
    theory  = ROOT.TGraph(N)       # theory
        
    
    upperLimit=[]
    
    for i in range(N):
        
        mass = values[i]
        
        limit = get_limits(f'LimitOutput/VLLTau_M{mass}.json',mass,SR)
        #print(mass, limit)
        
        median.SetPoint(    i,    values[i], limit[0]) # median
        unc.SetPoint(       i,    values[i], limit[1]) # s+ds
        unc.SetPoint( 2*N-1-i,    values[i], limit[2]) # s-ds
        theory.SetPoint(    i,    values[i], limit[3]) #theory
        
        
    upperLimit.extend([median,unc,theory])
    
    
    return upperLimit





def plot_limit(limit):    
    
    #Canvas
    W = 1000
    H = 800
    T = 0.08*H
    B = 0.12*H
    L = 0.15*W
    R = 0.25*W
    c = ROOT.TCanvas("c","c",100,100,W,H)
    c.SetFillColor(0)
    c.SetBorderMode(0)
    c.SetFrameFillStyle(0)
    c.SetFrameBorderMode(0)
    c.SetLeftMargin( L/W )
    c.SetRightMargin( R/W )
    c.SetTopMargin( T/H )
    c.SetBottomMargin( B/H )
    c.SetTickx(0)
    c.SetTicky(0)
    c.SetLogy(1)
    c.cd()
    
    ##frame
    frame = c.DrawFrame(1.4,0.001, 3.1, 10)
    frame.GetYaxis().CenterTitle()
    frame.GetYaxis().SetTitleSize(0.04)
    frame.GetXaxis().SetTitleSize(0.04)
    frame.GetXaxis().SetLabelSize(0.03)
    frame.GetYaxis().SetLabelSize(0.03)
    frame.GetXaxis().SetTitleOffset(1.2)
    frame.GetYaxis().SetTitleOffset(1.4)
    frame.GetXaxis().SetNdivisions(510)
    #frame.GetYaxis().CenterTitle(True)
    frame.GetYaxis().SetTitle("#sigma (pb)")
    frame.GetXaxis().SetTitle("m_{VLL} (GeV)")
    frame.SetMinimum(0.005)
    #frame.SetMaximum(max(up2s)*1.05)
    frame.SetMaximum(1000)
    frame.GetXaxis().SetLimits(50,300)
    
    
    x1 = 0.75
    x2 = x1 + 0.24
    y2 = 0.85
    y1 = 0.35
    legend =ROOT.TLegend(x1,y1,x2,y2)
    legend.SetFillStyle(0);legend.SetBorderSize(0);legend.SetTextSize(0.02);
    legend.SetTextAlign(12);legend.SetTextFont(42)
    #legend.SetHeader('95% CL upper limits')
    #legend.AddEntry(median,"observed",'L')
    #legend.AddEntry(unc   ,"Uncertainty",'f')
    
    for i,SR in enumerate(limit.keys()):
            
        median = limit[SR][0]
        unc    = limit[SR][1]    
    
        ##
        unc.SetFillColor(ROOT.kOrange)
        unc.SetLineColor(ROOT.kOrange)
        unc.SetFillStyle(1001)
    
        median.SetLineColor(i+1)
        median.SetLineWidth(2)
        median.SetLineStyle(1)
        median.SetMarkerStyle(20)
        median.SetMarkerSize(0.6)
        median.SetMarkerColor(i+1)
        
        if(i==0):
            median.Draw('LP')
            #unc.Draw('F')
        else:
            median.Draw('LPSAME')
            #unc.Draw('FSAME')
            
            
        legend.AddEntry(median,f"{SR}:{ATLAS_SR[SR]}",'L')    
    
    theory = limit[SR][2]
    legend.AddEntry(theory,"Theory xsec",'f')
    #theoryxsec
    theory.SetLineColor(ROOT.kRed)
    theory.SetFillColor(ROOT.kRed-9)
    theory.SetLineWidth(2)
    theory.SetLineStyle(1)
    theory.SetMarkerStyle(1)
    theory.Draw('L')
    
    ##
    ROOT.gPad.SetTicks(1,1)
    frame.Draw('sameaxis')
    

    legend.Draw()
    
    
    c.SaveAs("test_limit.pdf")
    c.Close()
    
    

In [5]:
values=[100,150,200,250]

SR=[f'SR{i}' for i in range (1,21)]

upperLimit={}
for sr in SR:
    upperLimit[sr] = get_UpperLimit(values,sr)

In [6]:
upperLimit

{'SR1': [<cppyy.gbl.TGraph object at 0xbbb2e60>,
 'SR2': [<cppyy.gbl.TGraph object at 0xbc30ea0>,
 'SR3': [<cppyy.gbl.TGraph object at 0xbb8d1a0>,
 'SR4': [<cppyy.gbl.TGraph object at 0xbc4d010>,
 'SR5': [<cppyy.gbl.TGraph object at 0xba6e550>,
 'SR6': [<cppyy.gbl.TGraph object at 0xbc5eac0>,
 'SR7': [<cppyy.gbl.TGraph object at 0xad42410>,
 'SR8': [<cppyy.gbl.TGraph object at 0xbba6700>,
 'SR9': [<cppyy.gbl.TGraph object at 0xbb9c990>,
 'SR10': [<cppyy.gbl.TGraph object at 0xbc38040>,
 'SR11': [<cppyy.gbl.TGraph object at 0xb814190>,
 'SR12': [<cppyy.gbl.TGraph object at 0x4b36c00>,
 'SR13': [<cppyy.gbl.TGraph object at 0xba70560>,
 'SR14': [<cppyy.gbl.TGraph object at 0xbc71690>,
 'SR15': [<cppyy.gbl.TGraph object at 0xbba94d0>,
 'SR16': [<cppyy.gbl.TGraph object at 0xbc569e0>,
 'SR17': [<cppyy.gbl.TGraph object at 0xbc27ac0>,
 'SR18': [<cppyy.gbl.TGraph object at 0xbb8eaa0>,
 'SR19': [<cppyy.gbl.TGraph object at 0xb807880>,
 'SR20': [<cppyy.gbl.TGraph object at 0xbbdfe10>,
  <cppyy.

In [7]:
plot_limit(upperLimit)

Info in <TCanvas::Print>: pdf file test_limit.pdf has been created


In [9]:
model = 'hst_VLEsing250_8TeV_200k.root'

model.split('_')[1]

'VLEsing250'